https://hesperia.gsfc.nasa.gov/rhessi3/data-access/rhessi-data/flare-list/index.html

http://sprg.ssl.berkeley.edu/~jimm/hessi/hsi_flare_list.html

http://sprg.ssl.berkeley.edu/%7Etohban/browser/?show=grth1+qlpcr+qlpds+qlpg9+qlifs+qli01+qli02+qli03+qli04+qli05+qli06+qlids&date=20020212&time=213530&bar=1
GOES classes: https://svs.gsfc.nasa.gov/10109

In [ ]:
import numpy as np
import scipp as sc

In [ ]:
da = sc.io.open_hdf5('data/hessi_flares.h5')

In [ ]:
da

In [ ]:
non_solar_id = da.attrs['origin_legend'].value['origin', sc.scalar('non_solar')].data.value
# da.masks['non_solar'] = (da.attrs['origin'] == non_solar_id)
solar_id = da.attrs['origin_legend'].value['origin', sc.scalar('solar')].data.value
da = da.groupby('origin').copy(solar_id)

In [ ]:
# da.masks['non_solar2'] = (da.coords['x'] == 0*sc.Unit('asec')) & (da.coords['y'] == 0*sc.Unit('asec'))
da = da[(da.coords['x'] != 0*sc.Unit('asec')) | (da.coords['y'] != 0*sc.Unit('asec'))]

In [ ]:
# da.masks['eclipsed'] = da.attrs['eclipsed']
da = da[~da.attrs['eclipsed']]

In [ ]:
da = da[da.attrs['min_energy'] >= 6.0 * sc.Unit('keV')]

In [ ]:
# manual edges to remove outliers -> non-solar?
b = sc.bin(da, edges=[sc.linspace('y', -1200, 1200, 100, unit='asec'), sc.linspace('x', -1200, 1200, 100, unit='asec')])

In [ ]:
fig = b.bins.sum().plot(aspect='equal')
fig.ax.set_xlabel('x [asec]')
fig.ax.set_ylabel('y [asec]')
fig

In [ ]:
sc.bin(da, edges=[sc.arange('time', da.coords['time'].min().value, da.coords['time'].max().value, 100000, unit='s')]).bins.sum().plot()

In [ ]:
xx = da.copy()
xx.data[...] = sc.scalar(1., unit='counts')

In [ ]:
sc.bin(xx, edges=[sc.arange('time', da.coords['time'].min().value, da.coords['time'].max().value, 3000000, unit='s')]).bins.sum().plot()